In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time



from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, SelectKBest, chi2, SelectKBest, mutual_info_classif
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, make_scorer, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder

In [ ]:
# change the names accordingly
df = pd.read_csv("SDSS_processed.csv")
df = df.drop('z', axis=1)
X = df.drop('Target', axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Preliminary Model Running 

Decision Tree, Random Forest, SVM, KNN, neural networks, and XGBoost

In [ ]:
def metrics(y_test, y_pred, y_scores):
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    n_classes = len(np.unique(y_test))
    precision_recall_auc = []
    for i in range(n_classes):
        y_test_binary = (y_test == i).astype(int)
        y_scores_binary = y_scores[:, i]
        precision_curve, recall_curve, _ = precision_recall_curve(y_test_binary, y_scores_binary)
        auc_score = auc(recall_curve, precision_curve)
        precision_recall_auc.append(auc_score)

    average_precision_recall_auc = np.mean(precision_recall_auc)

    return precision, recall, f1, average_precision_recall_auc

In [ ]:
n_runs = 20
precisions, recalls, f1s, auc_prs = [], [], [], []
total_time = 0

for _ in range(n_runs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    start_time = time.time()

    dt = DecisionTreeClassifier()
    dt.fit(X_train, y_train)
    y_pred_dt = dt.predict(X_test)
    y_scores_dt = dt.predict_proba(X_test)

    precision, recall, f1, auc_pr = metrics(y_test, y_pred_dt, y_scores_dt)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    auc_prs.append(auc_pr)

    total_time += time.time() - start_time

avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_auc_pr = np.mean(auc_prs)
avg_time = total_time / n_runs

avg_precision, avg_recall, avg_f1, avg_auc_pr, avg_time

In [ ]:
n_runs = 20
precisions_rf, recalls_rf, f1s_rf, auc_prs_rf = [], [], [], []
total_time_rf = 0

for _ in range(n_runs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    start_time_rf = time.time()

    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    y_scores_rf = rf.predict_proba(X_test)

    precision_rf, recall_rf, f1_rf, auc_pr_rf = metrics(y_test, y_pred_rf, y_scores_rf)
    precisions_rf.append(precision_rf)
    recalls_rf.append(recall_rf)
    f1s_rf.append(f1_rf)
    auc_prs_rf.append(auc_pr_rf)

    total_time_rf += time.time() - start_time_rf

avg_precision_rf = np.mean(precisions_rf)
avg_recall_rf = np.mean(recalls_rf)
avg_f1_rf = np.mean(f1s_rf)
avg_auc_pr_rf = np.mean(auc_prs_rf)
avg_time_rf = total_time_rf / n_runs

avg_precision_rf, avg_recall_rf, avg_f1_rf, avg_auc_pr_rf, avg_time_rf

In [ ]:
n_runs = 20
precisions_svm, recalls_svm, f1s_svm, auc_prs_svm = [], [], [], []
total_time_svm = 0

for _ in range(n_runs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    start_time_svm = time.time()

    svm = CalibratedClassifierCV(SVC())
    svm.fit(X_train, y_train)
    y_pred_svm = svm.predict(X_test)
    y_scores_svm = svm.predict_proba(X_test)

    precision_svm, recall_svm, f1_svm, auc_pr_svm = metrics(y_test, y_pred_svm, y_scores_svm)
    precisions_svm.append(precision_svm)
    recalls_svm.append(recall_svm)
    f1s_svm.append(f1_svm)
    auc_prs_svm.append(auc_pr_svm)

    total_time_svm += time.time() - start_time_svm

avg_precision_svm = np.mean(precisions_svm)
avg_recall_svm = np.mean(recalls_svm)
avg_f1_svm = np.mean(f1s_svm)
avg_auc_pr_svm = np.mean(auc_prs_svm)
avg_time_svm = total_time_svm / n_runs

avg_precision_svm, avg_recall_svm, avg_f1_svm, avg_auc_pr_svm, avg_time_svm

In [ ]:
n_runs = 20
precisions_knn, recalls_knn, f1s_knn, auc_prs_knn = [], [], [], []
total_time_knn = 0

for _ in range(n_runs):
    X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X, y, test_size=0.3)
    X_train_knn = X_train_knn.to_numpy()
    X_test_knn = X_test_knn.to_numpy()
    y_train_knn = y_train_knn.to_numpy()
    y_test_knn = y_test_knn.to_numpy()

    start_time_knn = time.time()

    knn = KNeighborsClassifier()
    knn.fit(X_train_knn, y_train_knn)
    y_pred_knn = knn.predict(X_test_knn)
    y_scores_knn = knn.predict_proba(X_test_knn)

    precision_knn, recall_knn, f1_knn, auc_pr_knn = metrics(y_test_knn, y_pred_knn, y_scores_knn)
    precisions_knn.append(precision_knn)
    recalls_knn.append(recall_knn)
    f1s_knn.append(f1_knn)
    auc_prs_knn.append(auc_pr_knn)

    total_time_knn += time.time() - start_time_knn

avg_precision_knn = np.mean(precisions_knn)
avg_recall_knn = np.mean(recalls_knn)
avg_f1_knn = np.mean(f1s_knn)
avg_auc_pr_knn = np.mean(auc_prs_knn)
avg_time_knn = total_time_knn / n_runs

avg_precision_knn, avg_recall_knn, avg_f1_knn, avg_auc_pr_knn, avg_time_knn

In [ ]:
n_runs = 20
precisions_xgb, recalls_xgb, f1s_xgb, auc_prs_xgb = [], [], [], []
total_time_xgb = 0

for _ in range(n_runs):
    X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y, test_size=0.3)

    start_time_xgb = time.time()

    xgb = XGBClassifier()
    xgb.fit(X_train_xgb, y_train_xgb)
    y_pred_xgb = xgb.predict(X_test_xgb)
    y_scores_xgb = xgb.predict_proba(X_test_xgb)

    precision_xgb, recall_xgb, f1_xgb, auc_pr_xgb = metrics(y_test_xgb, y_pred_xgb, y_scores_xgb)
    precisions_xgb.append(precision_xgb)
    recalls_xgb.append(recall_xgb)
    f1s_xgb.append(f1_xgb)
    auc_prs_xgb.append(auc_pr_xgb)

    total_time_xgb += time.time() - start_time_xgb

avg_precision_xgb = np.mean(precisions_xgb)
avg_recall_xgb = np.mean(recalls_xgb)
avg_f1_xgb = np.mean(f1s_xgb)
avg_auc_pr_xgb = np.mean(auc_prs_xgb)
avg_time_xgb = total_time_xgb / n_runs

avg_precision_xgb, avg_recall_xgb, avg_f1_xgb, avg_auc_pr_xgb, avg_time_xgb

Hyperparameter Tuning

In [ ]:
auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}
tree = DecisionTreeClassifier()
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring=auc_pr_scorer, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best AUC-PR score:", grid_search.best_score_)

In [ ]:
auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier()
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='f1_weighted', verbose=20)
grid_search_rf.fit(X_train, y_train)
print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best F1-weighted score for Random Forest:", grid_search_rf.best_score_)

In [ ]:
auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)
param_grid_svm = {
    'base_estimator__C': [0.001, 0.1, 1, 10],
    'base_estimator__kernel': ['linear', 'rbf', 'poly'],
    'base_estimator__gamma': ['scale', 'auto']
}

svm = CalibratedClassifierCV(SVC(), cv=3)
grid_search_svm = GridSearchCV(svm, param_grid_svm, cv=5, scoring=auc_pr_scorer, verbose=1, n_jobs=-1)
grid_search_svm.fit(X_train, y_train)

print("Best parameters for SVM:", grid_search_svm.best_params_)
print("Best AUC-PR score for SVM:", grid_search_svm.best_score_)

In [ ]:
auc_pr_scorer = make_scorer(average_precision_score, needs_proba=True)

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring=auc_pr_scorer, verbose=1, n_jobs=-1)
grid_search_knn.fit(X_train, y_train)

print("Best parameters for KNN:", grid_search_knn.best_params_)
print("Best AUC-PR score for KNN:", grid_search_knn.best_score_)

In [ ]:
param_grid_xgb = {
    'n_estimators': [50, 100, 300],
    'learning_rate': [0.01, 0.1, 0.5, 1],
    'max_depth': [None, 1, 5, 10],
    'reg_alpha': [0, 0.1, 1, 10],
    'reg_lambda': [0, 0.1, 1, 10]
}

xgb = XGBClassifier()
grid_search_xgb = GridSearchCV(xgb, param_grid_xgb, cv=3, scoring='f1_weighted', verbose=20)
grid_search_xgb.fit(X_train, y_train)

print("Best parameters for XGBoost:", grid_search_xgb.best_params_)
print("Best AUC-PR score for XGBoost:", grid_search_xgb.best_score_)

Feature Importance

In [ ]:
num_runs = 10
feature_importances = np.zeros((num_runs, X.shape[1]))

for i in range(num_runs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=300, reg_alpha=1, reg_lambda=10,
                              objective='multi:softmax', num_class=len(y.unique()))
    model.fit(X_train, y_train)
    feature_importances[i, :] = model.feature_importances_
    print(i)

avg_feature_importances = feature_importances.mean(axis=0)
sorted_idx = avg_feature_importances.argsort()[-20:]


In [ ]:
plt.figure(figsize=(8, 6))
plt.barh(range(14), avg_feature_importances[sorted_idx])
plt.yticks(range(14), X.columns[sorted_idx])
plt.xlabel('Average Feature Importance')
plt.title('Features Importance for XGBoost')
plt.show()

In [ ]:
X.columns

In [ ]:
num_runs = 10
feature_importances = np.zeros((num_runs, X_new.shape[1]))

for i in range(num_runs):
    X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=i)

    rf = RandomForestClassifier(n_estimators=200, max_depth=10, 
                                min_samples_split=4, min_samples_leaf=2, random_state=42)
    rf.fit(X_train, y_train)
    feature_importances[i, :] = rf.feature_importances_
    print(i)

# Average feature importances across all runs
avg_feature_importances_rf = feature_importances.mean(axis=0)
sorted_idx_rf = avg_feature_importances.argsort()[::-1][:9]

In [ ]:
sorted_idx_rfavg_feature_importances_rf = feature_importances.mean(axis=0)
sorted_idx_rf = sorted_idx_rfavg_feature_importances_rf.argsort()[::-1][:10]

In [ ]:
X_new.columns

In [ ]:
plt.figure(figsize=(8, 6))
plt.barh(range(9), avg_feature_importances_rf[sorted_idx_rf])
plt.yticks(range(9), X_new.columns[sorted_idx_rf])
plt.xlabel('Feature Importance')
plt.title('Features Importance for Random Forest')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top
plt.show()

In [ ]:
features_xgb = ['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z',
                'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z',
                'psf_u-g', 'psf_g-r', 'petro_u-g', 'petro_g-r']
features_rf = ['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z',
               'petroMag_r', 'petroMag_i', 'petroMag_z', 'psf_u-g']
all_features = set(features_rf + features_xgb)
combined_importances_rf = {feature: avg_feature_importances_rf[features_rf.index(feature)] if feature in features_rf else 0 
                           for feature in all_features}
combined_importances_xgb = {feature: avg_feature_importances[features_xgb.index(feature)] if feature in features_xgb else 0 
                            for feature in all_features}

# Sorting features based on the feature importance of XGBoost
sorted_features_xgb = sorted(all_features, key=lambda x: combined_importances_xgb[x], reverse=True)

# Creating arrays for plotting based on XGBoost sorted features
importances_rf_plot = [combined_importances_rf[feature] for feature in sorted_features_xgb]
importances_xgb_plot = [combined_importances_xgb[feature] for feature in sorted_features_xgb]

# Plotting
plt.figure(figsize=(10, 8))

indices = np.arange(len(sorted_features_xgb))

plt.barh(indices - bar_width/2, importances_xgb_plot, bar_width, label='XGBoost', color='#377eb8')
plt.barh(indices + bar_width/2, importances_rf_plot, bar_width, label='Random Forest', color='#4daf4a')

plt.yticks(indices, sorted_features_xgb)
plt.xlabel('Feature Importance')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)


plt.legend()
plt.gca().invert_yaxis()

plt.show()


In [ ]:
from sklearn.feature_selection import RFECV

xgb_model = XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=300, reg_aplha=1, reg_lambda=10)
rfecv = RFECV(estimator=xgb_model, step=1, cv=3, scoring='f1_weighted', verbose=20)
rfecv.fit(X, y)


In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'])
plt.show()

In [ ]:
rf_model_rd = RandomForestClassifier(n_estimators=200, max_depth=10, 
                                  min_samples_split=4, min_samples_leaf=2)
rfecv_rd = RFECV(estimator=rf_model_rd, step=1, cv=3, scoring='f1_weighted', verbose=20)
rfecv_rd.fit(X, y)

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'])
plt.show()


In [ ]:
import seaborn as sns
max_score_idx = np.argmax(rfecv.cv_results_['mean_test_score'])
max_score = rfecv.cv_results_['mean_test_score'][max_score_idx]
optimal_num_features = max_score_idx + 1 

max_score_idx_rf = np.argmax(rfecv_rd.cv_results_['mean_test_score'])
max_score_rf = rfecv_rd.cv_results_['mean_test_score'][max_score_idx_rf]
optimal_num_features_rf = max_score_idx_rf + 1

sns.set_theme(style="white")

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'], label='XGBoost', color='#377eb8')
plt.plot(range(1, len(rfecv_rd.cv_results_['mean_test_score']) + 1), rfecv_rd.cv_results_['mean_test_score'], label='Random Forest', color='#4daf4a')
plt.scatter(optimal_num_features, max_score, color='blue')
plt.scatter(optimal_num_features_rf, max_score_rf, color='green')
plt.annotate(f'Best XGBoost: {max_score:.4f}', (optimal_num_features, max_score), textcoords="offset points", xytext=(-50,-15), ha='center', color='#377eb8')
plt.annotate(f'Best RF: {max_score_rf:.4f}', (optimal_num_features_rf, max_score_rf), textcoords="offset points", xytext=(10,8), ha='center', color='#4daf4a')

plt.xlabel("Number of features selected")
plt.ylabel("F1 score")
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend()
plt.show()

In [ ]:
feature_names = X.columns

# Selected features
selected_features = feature_names[rfecv_rd.support_]

# Print selected features
print("Selected Features:", selected_features)

In [ ]:
from scipy import stats

# Parameters for the XGBoost model
params = {
    "learning_rate": 0.1,
    "max_depth": 10,
    "n_estimators": 300,
    "reg_alpha": 1,
    "reg_lambda": 10,
    "objective": "multi:softmax",
    "num_class": 3
}

num_simulations = 10

f1_scores = []
precision_scores = []
recall_scores = []
auc_pr_scores = []

for i in range(num_simulations):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)

    print(f"Simulation {i+1}")

    y_pred = model.predict(X_test)
    y_scores = model.predict_proba(X_test)

    precision, recall, f1, average_precision_recall_auc = metrics(y_test, y_pred, y_scores)

    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)
    auc_pr_scores.append(average_precision_recall_auc)

averaged_metrics = {
    "F1 Score": f1_scores,
    "Precision": precision_scores,
    "Recall": recall_scores,
    "AUC-PR": auc_pr_scores
}

averaged_results = {}

for metric, scores in averaged_metrics.items():
    average = np.mean(scores)
    ci_lower, ci_upper = stats.t.interval(0.95, len(scores)-1, loc=average, scale=stats.sem(scores))
    averaged_results[metric] = {"Average": average, "95% CI": (ci_lower, ci_upper)}

averaged_results



In [ ]:
df_new = df[['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z',
       'petroMag_r', 'petroMag_i', 'petroMag_z', 'psf_u-g', 'Target']]
X_new = df_new.drop('Target', axis=1)
y_new = df_new['Target']

In [ ]:
from sklearn.ensemble import RandomForestClassifier



rf_params = {
    "n_estimators": 200,
    "max_depth": 10,
    "min_samples_split": 4,
    "min_samples_leaf": 2
}

rf_f1_scores = []
rf_precision_scores = []
rf_recall_scores = []
rf_auc_pr_scores = []

for i in range(num_simulations):
    # Create and fit the model
    X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.3)
    rf_model = RandomForestClassifier(**rf_params)
    rf_model.fit(X_train, y_train)

    print(i)

    y_pred = rf_model.predict(X_test)
    y_scores = rf_model.predict_proba(X_test)

    precision, recall, f1, average_precision_recall_auc = metrics(y_test, y_pred, y_scores)

    rf_f1_scores.append(f1)
    rf_precision_scores.append(precision)
    rf_recall_scores.append(recall)
    rf_auc_pr_scores.append(average_precision_recall_auc)

rf_averaged_metrics = {
    "F1 Score": rf_f1_scores,
    "Precision": rf_precision_scores,
    "Recall": rf_recall_scores,
    "AUC-PR": rf_auc_pr_scores
}

rf_averaged_results = {}

for metric, scores in rf_averaged_metrics.items():
    average = np.mean(scores)
    ci_lower, ci_upper = stats.t.interval(0.95, len(scores)-1, loc=average, scale=stats.sem(scores))
    rf_averaged_results[metric] = {"Average": average, "95% CI": (ci_lower, ci_upper)}

rf_averaged_results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier
from scipy import interpolate


num_simulations = 10
n_classes = 3
common_recall_thresholds = np.linspace(0, 1, 100) 

precision_dict = {i: [] for i in range(n_classes)}

for i in range(num_simulations):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_scores = model.predict_proba(X_test)
    y_test_binarized = label_binarize(y_test, classes=range(n_classes))

    for class_id in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_test_binarized[:, class_id], y_scores[:, class_id])
        precision_interp = np.interp(common_recall_thresholds, recall[::-1], precision[::-1])
        precision_dict[class_id].append(precision_interp)

avg_precision = {class_id: np.mean(precision_dict[class_id], axis=0) for class_id in range(n_classes)}

colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

plt.figure(figsize=(8, 6))
for i, color in zip(range(n_classes), colors):
    plt.plot(common_recall_thresholds, avg_precision[i], color=color, lw=2,
             label=f'Class {i} (area = {auc(common_recall_thresholds, avg_precision[i]):0.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Average Precision-Recall Curve per Class')
plt.legend(loc="lower left")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier
from scipy import interpolate

params = {
    "learning_rate": 0.1,
    "max_depth": 10,
    "n_estimators": 300,
    "reg_alpha": 1,
    "reg_lambda": 10,
    "objective": "multi:softmax",
    "num_class": 3
}

num_simulations = 10
n_classes = 3
common_recall_thresholds = np.linspace(0, 1, 100) 

precision_dict = {i: [] for i in range(n_classes)}

for i in range(num_simulations):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_scores = model.predict_proba(X_test)
    y_test_binarized = label_binarize(y_test, classes=range(n_classes))

    for class_id in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_test_binarized[:, class_id], y_scores[:, class_id])
        precision_interp = np.interp(common_recall_thresholds, recall[::-1], precision[::-1])
        precision_dict[class_id].append(precision_interp)

avg_precision = {class_id: np.mean(precision_dict[class_id], axis=0) for class_id in range(n_classes)}

target_colors = {0: '#377eb8', 1: '#4daf4a', 2: '#e41a1c'}  # blue, green, red
class_names = {0: 'Star', 1: 'Galaxy', 2: 'Quasar'}

plt.figure(figsize=(8, 6))
for class_id, color in target_colors.items():
    plt.plot(common_recall_thresholds, avg_precision[class_id], color=color, lw=2,
             label=f'{class_names[class_id]} (area = {auc(common_recall_thresholds, avg_precision[class_id]):0.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)

plt.legend(loc="lower left")
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier
from scipy import interpolate

rf_params = {
    "n_estimators": 200,
    "max_depth": 10,
    "min_samples_split": 4,
    "min_samples_leaf": 2
}


num_simulations = 5
n_classes = 3
common_recall_thresholds = np.linspace(0, 1, 100) 

precision_dict = {i: [] for i in range(n_classes)}

for i in range(num_simulations):
    X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.3)
    model = RandomForestClassifier(**rf_params)
    model.fit(X_train, y_train)
    y_scores = model.predict_proba(X_test)
    y_test_binarized = label_binarize(y_test, classes=range(n_classes))

    for class_id in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_test_binarized[:, class_id], y_scores[:, class_id])
        precision_interp = np.interp(common_recall_thresholds, recall[::-1], precision[::-1])
        precision_dict[class_id].append(precision_interp)

avg_precision = {class_id: np.mean(precision_dict[class_id], axis=0) for class_id in range(n_classes)}

target_colors = {0: '#377eb8', 1: '#4daf4a', 2: '#e41a1c'}  # blue, green, red
class_names = {0: 'Star', 1: 'Galaxy', 2: 'Quasar'}

plt.figure(figsize=(8, 6))
for class_id, color in target_colors.items():
    plt.plot(common_recall_thresholds, avg_precision[class_id], color=color, lw=2,
             label=f'{class_names[class_id]} (area = {auc(common_recall_thresholds, avg_precision[class_id]):0.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)

plt.legend(loc="lower left")
plt.show()

